# Minimum Variance Beamformer

### 1. 生成随机数据(Normalized)

In [2]:
%matplotlib inline
import numpy as np
test_data = np.random.rand(10,5)
print(test_data)

[[0.44165064 0.13589517 0.3160139  0.95734931 0.70584854]
 [0.24513702 0.33953498 0.02633527 0.01353631 0.98432537]
 [0.61109763 0.31423796 0.61340871 0.18940292 0.17654108]
 [0.508474   0.97951417 0.07662894 0.75738817 0.46087482]
 [0.8782208  0.89941664 0.27477742 0.55923794 0.92744484]
 [0.54200348 0.86638297 0.07515069 0.84154677 0.16596313]
 [0.863227   0.20950782 0.68509641 0.92722771 0.14726707]
 [0.65724696 0.99691587 0.14795273 0.77204327 0.02246748]
 [0.39205626 0.18273275 0.71251297 0.27630306 0.33365591]
 [0.03099494 0.51657958 0.29185079 0.94167627 0.59857266]]


### 2. 生成子组

In [3]:
subset_size = 3
subsets = list()
for i in range(test_data.shape[1]):
    if i + subset_size <= test_data.shape[1]:
        subset = test_data[:,i:i+subset_size]
        subsets.append(subset)
print(subsets)

[array([[0.44165064, 0.13589517, 0.3160139 ],
       [0.24513702, 0.33953498, 0.02633527],
       [0.61109763, 0.31423796, 0.61340871],
       [0.508474  , 0.97951417, 0.07662894],
       [0.8782208 , 0.89941664, 0.27477742],
       [0.54200348, 0.86638297, 0.07515069],
       [0.863227  , 0.20950782, 0.68509641],
       [0.65724696, 0.99691587, 0.14795273],
       [0.39205626, 0.18273275, 0.71251297],
       [0.03099494, 0.51657958, 0.29185079]]), array([[0.13589517, 0.3160139 , 0.95734931],
       [0.33953498, 0.02633527, 0.01353631],
       [0.31423796, 0.61340871, 0.18940292],
       [0.97951417, 0.07662894, 0.75738817],
       [0.89941664, 0.27477742, 0.55923794],
       [0.86638297, 0.07515069, 0.84154677],
       [0.20950782, 0.68509641, 0.92722771],
       [0.99691587, 0.14795273, 0.77204327],
       [0.18273275, 0.71251297, 0.27630306],
       [0.51657958, 0.29185079, 0.94167627]]), array([[0.3160139 , 0.95734931, 0.70584854],
       [0.02633527, 0.01353631, 0.98432537],
     

### 3. 生成滑动窗口

In [13]:
window_size = 5
window_step = 1
for i in range(test_data.shape[0]):
    subset_cov_sum = None
    print("Step: %s" % i)
    for j in range(len(subsets)):
        if i + window_size <= test_data.shape[0]:
            # 1. Get raw subset
            print("Subset - %s" % j)
            print("*** Raw ***")
            subset_raw = subsets[j][i:i+window_size,]
            print(subset_raw)
            # 2. Calculate covariance matrix
            print("*** Cov ***")
            subset_cov = np.cov(subset_raw, rowvar=False)
            print(subset_cov)
            if subset_cov is None:
                break
            if subset_cov_sum is None:
                subset_cov_sum = subset_cov
            else:
                subset_cov_sum += subset_cov
    print("------------------Step %s------------------" % i)
    # 3. Calculate covariance matrix mean (Element-wise)
    print("*** Cov mean ***")
    if subset_cov_sum is None:
        break
    allset_cov_mean = subset_cov_sum/len(subsets)
    print(allset_cov_mean)
    # 4. Calculate the inverse of covariance matrix mean
    print("*** Inverse Cov mean ***")
    allset_cov_mean_inverse = np.linalg.inv(allset_cov_mean)
    print(allset_cov_mean_inverse)
    # 5. Calculate weight
    print("*** Weight ***")
    allset_cov_mean_inverse_rowsum = np.sum(allset_cov_mean_inverse, axis=1)
    print(allset_cov_mean_inverse_rowsum)
    print("------------------Step %s------------------" % i)

Step: 0
Subset - 0
*** Raw ***
[[0.44165064 0.13589517 0.3160139 ]
 [0.24513702 0.33953498 0.02633527]
 [0.61109763 0.31423796 0.61340871]
 [0.508474   0.97951417 0.07662894]
 [0.8782208  0.89941664 0.27477742]]
*** Cov ***
[[ 0.05425282  0.04760281  0.02482944]
 [ 0.04760281  0.14415282 -0.03270452]
 [ 0.02482944 -0.03270452  0.05411688]]
Subset - 1
*** Raw ***
[[0.13589517 0.3160139  0.95734931]
 [0.33953498 0.02633527 0.01353631]
 [0.31423796 0.61340871 0.18940292]
 [0.97951417 0.07662894 0.75738817]
 [0.89941664 0.27477742 0.55923794]]
*** Cov ***
[[ 0.14415282 -0.03270452  0.0292737 ]
 [-0.03270452  0.05411688 -0.00419237]
 [ 0.0292737  -0.00419237  0.15298427]]
Subset - 2
*** Raw ***
[[0.3160139  0.95734931 0.70584854]
 [0.02633527 0.01353631 0.98432537]
 [0.61340871 0.18940292 0.17654108]
 [0.07662894 0.75738817 0.46087482]
 [0.27477742 0.55923794 0.92744484]]
*** Cov ***
[[ 0.05411688 -0.00419237 -0.05088587]
 [-0.00419237  0.15298427 -0.005565  ]
 [-0.05088587 -0.005565    0.1

### 计算R(n)的估计
$ \hat{R}(n)=\frac{1}{(2K+1)(M-L+1)}\sum_{k=-K}^K\sum_{l=0}^{M-L}\bar{X_l}(n-k)\bar{X_l}(n-k)^{H} $